# 城市热点汇总 Notebook

该 Notebook 演示如何使用新的 `spdatalab.dataset.bbox` 模块来整理 bbox 热点分析的结果。


In [ ]:
from spdatalab.analysis.notebook_support import (
    NB_FAST_ENV_VAR,
    configure_project_paths,
    display_config,
    is_fast_mode,
    preview_dataframe,
)
from spdatalab.dataset.bbox.summary import BatchTop1Config, BBoxHotspotBatch

from sqlalchemy import create_engine
import os
import pandas as pd

ctx = configure_project_paths()
display_config(ctx)

LOCAL_DSN = os.environ.get("LOCAL_DSN")
if not LOCAL_DSN:
    print("未设置 LOCAL_DSN 环境变量，如需连接数据库请先配置。")

FAST_MODE = is_fast_mode()
print(f"Fast mode: {FAST_MODE} (设置 {NB_FAST_ENV_VAR}=1 可启用测试模式)")

engine = None
if not FAST_MODE and LOCAL_DSN:
    engine = create_engine(LOCAL_DSN, future=True)


In [ ]:
config = BatchTop1Config(
    output_table="city_top1_hotspots",
    max_cities=3 if FAST_MODE else None,
    top_percent=1.0,
)

display_config({
    "output_table": config.output_table,
    "max_cities": config.max_cities,
    "top_percent": config.top_percent,
})


In [ ]:
if engine is not None:
    runner = BBoxHotspotBatch(engine)
    inspection = runner.inspect(config)
    display_config({
        "analysis_date": inspection.analysis_date,
        "available_cities": inspection.available_cities,
        "expected_rows": inspection.expected_rows,
    })

    if inspection.available_cities:
        result = runner.run(config)
        display_config({
            "successful_cities": result.successful_cities,
            "failed_cities": result.failed_cities,
            "rows_inserted": result.extracted_rows,
            "elapsed_seconds": f"{result.elapsed_seconds:.2f}",
        })
else:
    sample = pd.DataFrame(
        [
            {"city_id": "A001", "bbox_count": 12, "scene_count": 4},
            {"city_id": "B009", "bbox_count": 7, "scene_count": 3},
        ]
    )
    print("使用示例数据预览热点结果：")
    preview_dataframe(sample)
